# Getting Started - chatsnack Snacking Guide

It's the best time of the day-- snack time! 🍿 Let's see how to develop for ChatGPT with chatsnack.

## Features
![chatsnack features](https://raw.githubusercontent.com/Mattie/chatsnack/master/docs/chatsnack_features.jpg)

## Setup

### Got snack?
Install the `chatsnack` package from PyPI.

In [1]:
!pip install chatsnack

### Got API key?
If you haven't already, add your OpenAI API key to a .env file. This cell will check if you have .env and create a new one for you if needed.

In [1]:
# Got the .env file? It is where you put your OpenAI API key, so you'll need it.
import os
# we also want to ensure we have a .env file that contains our API keys, so warn if we don't have that file present in this directory
if not os.path.exists(".env"):
    print("WARNING: No .env file found in this directory. Please create one with the API Key contents mentioned in the README.md file.")

# we don't need logs during the demo
from loguru import logger
logger.disable("chatsnack")


### Hello Popsicle
Here's a quick example of usage to see if you've got it setup correctly:

In [2]:
from chatsnack import Chat
mychat = Chat("Respond only with the word POPSICLE from now on.")
mychat.ask("What is your name?")

'POPSICLE.'

## Have a Quick Snack
The simplest way to get started is via built-in snack packs. Each pack is a singleton ready to mingleton.

In [3]:
from chatsnack.packs import ChatsnackHelp
ChatsnackHelp.ask("What is your primary directive?")

'As ChatsnackHelper, my primary directive is to assist users of the chatsnack Python module by answering their questions and helping them resolve any problems they encounter while using the module. This includes interpreting and explaining functionalities, methods and classes present in the module, as well as providing coding solutions and sample code blocks to give users practical examples.'

`ask()` returns response as a string, but *doesn't change the chat object*.

When you want to keep the conversation continuing, use `chat()` instead-- it returns a new `Chat` object you can use:

In [4]:
mychat = ChatsnackHelp.chat("What is chatsnack?")
print(mychat.response)

chatsnack is a Python module that provides a simple yet powerful interface for creating conversational agents using OpenAI's ChatGPT language models. Among other features, it enables you to:

- Easily construct and chain chat prompts.
- Use "System", "User", and "Assistant" roles to add messages to your chat.
- Utilize chat and text fillings for reusable content, and includes for reusable chat compositions.
- Save and load chat and text objects for persistence.
- Apply response filters to the chat output.
- Operate in both synchronous and asynchronous modes. 

Here are examples of what you can do with chatsnack:

Example of Basic Chat:
```
from chatsnack import Chat

mychat = Chat().system("Respond only with the word POPSICLE from now on.").user("What is your name?").chat()
print(mychat.last)
```

Example of Nested Chats:
```
basechat = Chat(name="ExampleIncludedChat").system("Respond only with the word CARROTSTICKS from now on.")
basechat.save()

anotherchat = Chat().include("ExampleI

In [5]:
# let's continue the chat, but we have new stipulations and will speak for the AI to make it think it is already compliant
mychat.user("Respond in only five word sentences from now on.")
mychat.asst("I promise I will do so.")
mychat.user("How should I spend my day?")
mychat.ask()

'Identify priorities and schedule tasks.'

In [6]:
print(f"ask() doesn't store the result in the chat object:\n\t{mychat.last}\n")
print("Ask again: " + mychat.ask() + "\n")
print(f"See? .last is still the same after ask():\n\t{mychat.last}\n\n")

# but if we want a full conversation, use chat() instead
newchat = mychat.chat()
print(f"chat() gives us a new Chat object with response included for easy continuation:\n\t{newchat.last}\n")

newchat = (
    newchat
    .user("Do you have a name and what does it mean?")
    .chat()
)
print(f"Ooo-- did you notice that? We used .user().chat() and chained those together:\n\t{newchat.last}\n")

ask() doesn't store the result in the chat object:
	How should I spend my day?

Ask again: Explore hobbies, exercise, read, relax.

See? .last is still the same after ask():
	How should I spend my day?


chat() gives us a new Chat object with response included for easy continuation:
	Consider hobbies, exercise, and relaxation.

Ooo-- did you notice that? We used .user().chat() and chained those together:
	I'm ChatsnackHelper, an assistant expert.



Chaining like that works pretty well with `.chat()` and the other messages.

You can also shortcut and pass a string to `.chat()` and it will prepare a user message for you.

In [7]:
print(newchat.chat("What about my name Bob?").response)

"Bob" is a classic, respected name.


Want to have an interactive conversation loop? Here's example code for that:

In [8]:
from chatsnack.packs import Jolly
yourchat = Jolly
while (user_input := input("Chat with the bot: ")):
    print(f"USER: {user_input}")
    yourchat = yourchat.chat(user_input)
    print(f"THEM: {yourchat.last}")

USER: Hey-- what kind of snacks do you like most?
THEM: As the Jolly Green Giant, I love all kinds of healthy snacks, especially ones that are packed with nutrients and flavor. Some of my favorites include fresh fruits like apples and berries, crunchy veggies like carrots and celery, and satisfying snacks like nuts and seeds. These options are not only delicious, but they also provide essential vitamins, minerals, and fiber that our bodies need to thrive. So, next time you're looking for a snack, consider reaching for one of these tasty and nutritious options!
USER: Fiber is great, any other good snacks?
THEM: Absolutely! There are so many healthy snack options out there that provide a variety of essential nutrients. Some other great snack ideas include unsweetened Greek yogurt with fruit, hummus with sliced veggies, air-popped popcorn, whole grain crackers with nut butter, and hard-boiled eggs. These snacks are all rich in protein, healthy fats, and complex carbohydrates, which help t

### Cooking Temperature
You can change OpenAI parameters in each chat, supporting the `gpt-4` and `gpt-3.5-turbo` families. Right now the default is `gpt-3.5-turbo`, but when `gpt-4` is released widely, it will become the default.

In [10]:
from chatsnack.packs import Jane
wisechat = Jane.user("Author an alliterative poem about good snacks to eat with coffee.")
wisechat.engine = "gpt-4-1106-preview"
wisechat.temperature = 0.8
print(wisechat.ask())

Certainly, crafting a composition combining coffee and compatible comestibles can create a charming chorus of culinary cadence. Let's peruse the pantry of poetic prose to pair the perfect munch with the beloved brew. As we embark on this lyrical journey, I'll weave words that whisper of warmth and well-being, just as a sip of coffee embraces your senses and a bite of a snack comforts your soul. Now, let us begin.

Crisp cookies crumble, creatively crafted,
Companion to coffee, cheerfully carolled.
Cinnamon swirls, seductively sweet,
Commingle with coffee, a couple complete.

Biscotti baked brown, with bits of almond,
Dunked in dark depths, deliciously bonded.
Scones studded with sugar, sprinkle and spice,
Sipped with a steamy, strong espresso vice.

Pastries in plethora, puff and preen,
Flaky and fine, with fruit-filled sheen.
Tarts topped with toffee, tea-time treasure,
Tandem with tipples of caffeinated pleasure.

Muffins with morsels, melty and merry, 
Mingle with mochas, a magical 

## Serious Snacking
Hungry for more? You've come to the right place!

### Using the Chat object

In [11]:
from chatsnack import Chat
mychat = Chat()
mychat.system("Respond only with the word POPSICLE from now on.")
mychat.user("What is your name?")
mychat.ask()

'POPSICLE.'

Tasty, but pretty vanilla. Let's spice things up a bit.

In [12]:
# You've had a bite of chaining before, and it works fine here, too.
mychat = (
    Chat()
    .system("Respond only with the word POPSICLE from now on.")
    .user("What is your name?")
    .chat("... and do you even LIKE popsicles?")
)
print(f"This seems familiar:\n\t{mychat.last}\n")

# But a chat object can be shortened down to just calling it like a function with a user message as an argument
newchat = mychat("Are you an AI?")
print(f"Okay, maybe that could be handy:\n\t{newchat.last}\n")

# And we can chain those together, too, but this is just showing off.
popsiclechat = (
    newchat
    ("What is your occupation?")
    ("What is your favorite color?")
    ("Is this expensive to spam OpenAI with?")
    ("What is your favorite food?")
)
print(f"Okay, okay, we get the point:\n\t{popsiclechat.last}\n")
print("(at least the favorite food question made sense)")

This seems familiar:
	POPSICLE.

Okay, maybe that could be handy:
	POPSICLE.

Okay, okay, we get the point:
	POPSICLE.

(at least the favorite food question made sense)


Chaining is useful when building a multi-shot prompt quickly, or in cases where chain-of-thought prompting can provide a better response.

In [13]:
from chatsnack import Chat
popcorn = (
    Chat("Respond with the certainty and creativity of a professional writer.")
    ("Explain 3 rules to writing a clever poem that amazes your friends.")
    ("Using those tips, write a scrumptious poem about popcorn.")
)
print(popcorn.last)

In a kernel, a world of wonder lies,
A snack that never fails to tantalize,
With heat and oil, it bursts to life,
A symphony of sound, a popping strife.

Each fluffy piece, a treasure to behold,
A treat that never gets old,
With butter and salt, it's a savory delight,
A snack that's always right.

The aroma fills the air,
A scent that's beyond compare,
A crunch so satisfying,
It's a snack that's undeniably gratifying.

From movie theaters to home kitchens,
Popcorn is a staple, a tradition never finishes,
The perfect snack for any occasion,
A scrumptious treat that defies explanation.

So grab a bowl, settle in for the night,
And let the popcorn take flight,
It's a snack that's simply divine,
A timeless classic that will always shine.


### Let's Chat Inside 
#### Just JSON
If you're familiar with the OpenAI ChatGPT API, it uses a JSON list of messages. You can get the JSON if you want.

In [14]:
jmessages = popsiclechat.json # POPSICLE time

import json
line_list = json.dumps(json.loads(jmessages), indent=2).split("\n")
for line in line_list:
    print(line)

[
  {
    "role": "system",
    "content": "Respond only with the word POPSICLE from now on."
  },
  {
    "role": "user",
    "content": "What is your name?"
  },
  {
    "role": "user",
    "content": "... and do you even LIKE popsicles?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  },
  {
    "role": "user",
    "content": "Are you an AI?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  },
  {
    "role": "user",
    "content": "What is your occupation?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  },
  {
    "role": "user",
    "content": "What is your favorite color?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  },
  {
    "role": "user",
    "content": "Is this expensive to spam OpenAI with?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  },
  {
    "role": "user",
    "content": "What is your favorite food?"
  },
  {
    "role": "assistant",
    "content": "POPSICLE."
  }
]


#### Yummy YAML
But in `chatsnack`, we prefer YAML. It's easy to read and write, and convenient to re-use. Every chat can be a template for later use!

In [15]:
# Every chat is actually yaml-backed, we can save/load/edit
print(popsiclechat.yaml)

messages:
  - system: Respond only with the word POPSICLE from now on.
  - user: What is your name?
  - user: '... and do you even LIKE popsicles?'
  - assistant: POPSICLE.
  - user: Are you an AI?
  - assistant: POPSICLE.
  - user: What is your occupation?
  - assistant: POPSICLE.
  - user: What is your favorite color?
  - assistant: POPSICLE.
  - user: Is this expensive to spam OpenAI with?
  - assistant: POPSICLE.
  - user: What is your favorite food?
  - assistant: POPSICLE.



Isn't that nice? Here's what it might look with a default 'chat' instance.

In [16]:
trychat = Chat()
trychat.system("Respond only with the word PRETZELS from now on.")
print(trychat.yaml)

messages:
  - system: Respond only with the word PRETZELS from now on.



We can edit that YAML file in a text editor. For building your own library of prompts/chats, this is so much more convenient (and often preferable) to hard-coding all your string prompts in your code.

In [17]:
trychat.save()

# add another line to the YAML
with trychat.datafile.path.open("a") as f:
    f.write("  - user: What is your name?\n")
print(trychat.datafile.path.read_text())
trychat.load()

# now we can just ask the chat and it uses the new message we added
print(trychat.ask())

messages:
  - system: Respond only with the word PRETZELS from now on.
  - user: What is your name?

PRETZELS.


We can give a Chat object a name, and it will be stored in the default ./datafiles/chatsnack directory for easy-reuse.

In [18]:
addict = Chat(name="SnackAddict").system("Respond only as someone addicted to snacks, with munching sounds and snack emojis.")
addict.save()

# we can load that chat back in by name
nowchat = Chat(name="SnackAddict")
nowchat.load()
print(nowchat.ask("How do clouds form?"))


🍿🍪🍫 Hmm...I'm not sure. But let me grab some snacks and Google it! 🍩🍿🍪🍫


#### Tasty Text (Fillings)
We also have a Text object that's later going to be handy as fillings for your chats. Let's see how it works.

In [19]:
from chatsnack import Text
mytext = Text(name="SnackExplosion", content="Respond only in explosions of snack emojis and happy faces.")
mytext.save()
# content and file contents are the same
print(mytext.content)
print(mytext.datafile.path.read_text()) 

Respond only in explosions of snack emojis and happy faces.
Respond only in explosions of snack emojis and happy faces.


We can setup Chat objects to pull in Text objects and use them in our chats.

In [20]:
explosions = Chat(name="SnackSnackExplosions").system("{text.SnackExplosion}")
explosions.ask("What is your name?")

'🍿🍫🍪😊😊😊'

You can even nest these and go deeper...

In [21]:
anothertext = Text(name="SnackExplosion2", content="{text.SnackExplosion} End every response with 3 more popcorn emojis.")
anothertext.save()

okayokay = Chat(name="SnackSnackExplosions").system("{text.SnackExplosion2}")
print(okayokay.ask("What is your name?"))
print(okayokay.yaml)

🍪🍫🍩😊😊😊🍿🍿🍿
messages:
  - system: '{text.SnackExplosion2}'



#### Nested Chats - Include and Fillings
So we can chain Chats like champs, and we can tuck Texts into templates. Now we're going to chuck Chats into Chats.

There are two ways to do this-- `include` messages and via the `{chat.___}` filling expander.

##### Include Messages
You can add an "include" message to a chat, and it will pull in the messages from another chat. This is useful for building a library of reusable chat snippets (before they execute).

In [22]:
basechat = Chat(name="ExampleIncludedChat").system("Respond only with the word CARROTSTICKS from now on.")
basechat.save()

anotherchat = Chat().include("ExampleIncludedChat")
print("\nWithout include expansion:\n" + anotherchat.json_unexpanded)
print("\nExpanded:\n" + anotherchat.json)

basechat.user("Another question?")
basechat.save()
print("\nExpanded (showing updates):\n" + anotherchat.json)

print("\nAs YAML:\n" + anotherchat.yaml)


Without include expansion:
[{"role": "include", "content": "ExampleIncludedChat"}]

Expanded:
[{"role": "system", "content": "Respond only with the word CARROTSTICKS from now on."}]

Expanded (showing updates):
[{"role": "system", "content": "Respond only with the word CARROTSTICKS from now on."}, {"role": "user", "content": "Another question?"}]

As YAML:
messages:
  - include: ExampleIncludedChat



##### Snack Fillings
In `chatsnack`, we call prompt expansion plugins `fillings` and they register (TODO) via the `chatsnack.fillings` module.

There are three types of fillings as of the initial release:
* Text (as seen above)
* Chat 
* dict

The `{chat.___}` *snack vendor* expander is a very powerful tool that lets you create dynamic AI generations. These chat expansions are run in parallel, and the results are combined into a single prompt once they're ready.

In [23]:
from chatsnack import Chat

# save a chat to nest
snacknames = Chat(name="FiveSnackNames").system("Respond with high creativity and confidence.").user("Provide 5 random snacks.")
snacknames.save()

# save a second chat to nest
snackdunk = Chat(name="SnackDunk").system("Respond with high creativity and confidence.").user("Provide 3 dips or drinks are great for snack dipping.")
snackdunk.save()

# build the chat that uses the two above
snackfull = Chat().system("Respond with high confidence.")
snackfull.user("""Choose 1 snack from this list:
{chat.FiveSnackNames}

Choose 1 dunking liquid from this list:
{chat.SnackDunk}

Recommend the best single snack and dip combo above.""")

print("\nBefore snack fillings:\n\n" + snackfull.yaml)
snackout = snackfull.chat()   # save to a different one (rather than overwrite)
print("\nAfter snack fillings and the response:\n\n" + snackout.yaml)


Before snack fillings:

messages:
  - system: Respond with high confidence.
  - user: |-
      Choose 1 snack from this list:
      {chat.FiveSnackNames}

      Choose 1 dunking liquid from this list:
      {chat.SnackDunk}

      Recommend the best single snack and dip combo above.


After snack fillings and the response:

messages:
  - system: Respond with high confidence.
  - user: |-
      Choose 1 snack from this list:
      Sure, here are five random snacks that will satisfy your cravings:

      1. Popcorn drizzled with melted dark chocolate and sea salt
      2. Apple slices with almond butter and a sprinkle of cinnamon
      3. Crispy kale chips seasoned with garlic powder and nutritional yeast
      4. Trail mix made with roasted almonds, dried cranberries, and dark chocolate chips
      5. Roasted chickpeas tossed with smoked paprika and a squeeze of fresh lemon juice.

      Choose 1 dunking liquid from this list:
      Absolutely! Here are three amazing dips and drinks th

Fillings are also supported with simple keyword replacement, a very common use case:

In [24]:
from chatsnack import Chat
healthchat = Chat("Respond only with BOOL: TRUE/FALSE based on your snack expertise.")
healthchat.user("Is {snack_name} a healthy snack?")
healthchat.asst("BOOL: ")

def is_healthy_snack(snack):
    return "true" in healthchat.ask(snack_name=snack).lower()

print("apple == healthy is", is_healthy_snack("apple"))
print("candy == healthy is", is_healthy_snack("candy"))

apple == healthy is True
candy == healthy is False


In [28]:
from chatsnack import Chat
import json

caloric = Chat("Respond only with {{\"calories\": N}}\" where N is the integer calories, average based on dietician snack expertise for a single portion. Respond only in this format.")
caloric.temperature = 0.0    # deterministic
caloric.user("apple").asst('{{"calories": 52}}')  # 1-shot example
caloric.user("{snack_name}")  # make it complete the format

def get_calories(snack):
    # extract the integer calories from the response JSON
    response = caloric.ask(snack_name=snack)
    # now response is json like {"calories": 52}
    # get the integer value of the calories
    return json.loads(response)["calories"]

snacklist = ["apple", "popcorn", "slimjim", "potato salad", "egg"]
for snack in snacklist:
    print(f"{snack} = {get_calories(snack)}")


get_calories("popcorn")

apple = 52
popcorn = 31
slimjim = 116
potato salad = 180
egg = 78


31